In [1]:
! find . -name "*.pyc" -exec rm -f {} \;

In [2]:
# pytorch something
# numpy for example

import numpy as np
from regr.utils import extract_args


def np_debug(mat, show_value=False):
    args = extract_args(_stack_back_level_=1)
    for k, v in args.items():
        v = np.array(v)
        print('{}: {}, {}'.format(k, v.shape, v.dtype))
        if show_value:
            print(v)


def mask_expand(mask_len, max_len):
    mask_len = np.array(mask_len)
    expanded = np.tile(np.expand_dims(
        np.arange(max_len), axis=0), mask_len.shape + (1,))
    return expanded < np.expand_dims(mask_len, axis=-1)


def softmax(x): return np.exp(x) / np.exp(x).sum(axis=-1, keepdims=True)


# config and data
batch_size = 2
max_len = 10
max_phrase = 5
mask_len = np.random.randint(max_len / 4, max_len, size=batch_size)
np_debug(mask_len)
repr_size = 16

entity_types = 3
relation_types = 2

mask = mask_expand(mask_len, max_len)
np_debug(mask)  # (batch, word,)

# belief layer
bword = np.ones((batch_size, max_len)) * mask
np_debug(bword)  # (batch, word,)
# embedding layer
rword = np.random.random((batch_size, max_len, repr_size)) * \
    np.expand_dims(mask, axis=-1)  # (batch, batch, repr)
np_debug(rword)

# links
# phrase -.-> word
tokenizor = np.random.random(
    (batch_size, max_phrase, max_len)) * np.expand_dims(mask, axis=1)
tokenizor = (tokenizor == tokenizor.max(axis=1, keepdims=True)
             ) & np.expand_dims(mask, axis=1)
np_debug(tokenizor)  # (batch, prhase, word,)

phrase_mask = tokenizor.sum(axis=-1).astype(np.bool)
np_debug(phrase_mask)  # (batch, prhase,)

# sentence -.-> phrase
sentence_rel = np.ones((batch_size, max_phrase), dtype=np.bool) * phrase_mask
np_debug(sentence_rel)  # (batch, prhase,)

# entity ---> phrase
entity_pred = np.random.choice(
    [False, True], (batch_size, max_phrase)) * phrase_mask
np_debug(entity_pred)  # (batch, prhase,)

# people|organization|other ---> entity
entity_clas = np.random.randn(batch_size, max_phrase, entity_types)
entity_clas = softmax(entity_clas) * np.expand_dims(phrase_mask, axis=-1)
np_debug(entity_clas)  # (batch, prhase, entity_type,)

relation_mask = np.matmul(np.expand_dims(
    phrase_mask, axis=2), np.expand_dims(phrase_mask, axis=1))
np_debug(relation_mask)  # (batch, prhase, prhase,)

# work_for ---> pair
relation_clas = np.random.randn(
    batch_size, max_phrase, max_phrase, relation_types)
relation_clas = softmax(relation_clas) * np.expand_dims(relation_mask, axis=-1)
np_debug(relation_clas)  # (batch, prhase, prhase, relation_type,)


mask_len: (2,), int64
mask: (2, 10), bool
bword: (2, 10), float64
rword: (2, 10, 16), float64
tokenizor: (2, 5, 10), bool
phrase_mask: (2, 5), bool
sentence_rel: (2, 5), bool
entity_pred: (2, 5), bool
entity_clas: (2, 5, 3), float64
relation_mask: (2, 5, 5), bool
relation_clas: (2, 5, 5, 2), float64


In [3]:
from regr.graph import Graph
from regr.entity import Entity

Graph.clear()
Entity.clear()

with Graph('global') as graph:
    with Graph('linguistic') as ling_graph:
        # linguistic layer
        word = Entity(name='word', blf=bword, rpt=rword)
        phrase = Entity(name='phrase')
        sentence = Entity(name='sentence')
        phrase.hasa(word, rel=tokenizor)
        sentence.hasa(phrase, rel=sentence_rel)

    with Graph('application') as app_graph:
        # application nodes
        entity = Entity(name='entity', blf=entity_pred)
        entity.isa(phrase)
        pair = Entity(name='pair')
        pair.isa((entity, entity))

        people = Entity(name='people', blf=entity_clas[:,:,0])
        organization = Entity(name='organization', blf=entity_clas[:,:,1])
        other = Entity(name='other', blf=entity_clas[:,:,2])
        people.isa(entity)
        organization.isa(entity)
        other.isa(entity)

        work_for = Entity(name='work-for', blf=relation_clas[:,:,:,0])
        work_for.isa({'employee':people, 'employer':organization})
        work_for.isa(pair)

graph

Graph(name='global', what={'entities': [],
 'subgraphs': [Graph(name='linguistic', what={'entities': [Entity(name='word', what={'has-a': {},
 'is-a': {}}),
              Entity(name='phrase', what={'has-a': {},
 'is-a': {}}),
              Entity(name='sentence', what={'has-a': {},
 'is-a': {}})],
 'subgraphs': [],
 'super': Graph(name='global')}),
               Graph(name='application', what={'entities': [Entity(name='entity', what={'has-a': {},
 'is-a': {Entity(name='phrase'): [isa(src=Entity(name='entity'), dst=Entity(name='phrase'), offset=1, name=None)]}}),
              Entity(name='pair', what={'has-a': {},
 'is-a': {Entity(name='entity'): [isa(src=Entity(name='pair'), dst=Entity(name='entity'), offset=1, name='arg-0'),
                                  isa(src=Entity(name='pair'), dst=Entity(name='entity'), offset=2, name='arg-1')]}}),
              Entity(name='people', what={'has-a': {},
 'is-a': {Entity(name='entity'): [isa(src=Entity(name='people'), dst=Entity(name='entity

In [4]:
people()
organization()

evaluating work-for(employee,1):(5, 10) ---is-a--> people:(10,)
    distance = 2.29716346511
evaluating work-for(employer,2):(5, 10) ---is-a--> organization:(10,)
    distance = 2.76692599357


2.7669259935650574

In [5]:
Graph.get('application')()

evaluating other(None,1):(1, 10) ---is-a--> entity:(10,)
    distance = 1.80253576702
evaluating organization(None,1):(1, 10) ---is-a--> entity:(10,)
    distance = 1.91963780692


ValueError: axes don't match array